# Import Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd
import logging

In [2]:
data=pd.read_csv('Preprocessed_data.csv')

#Dividing target variable from the main dataset

X = data.iloc[: , 0:-1]
Y = data.iloc[: , -1] 

In [3]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1)

# Encoding categorical data
## Encoding the Independent Variable

In [4]:
ordinal_encoder = OrdinalEncoder()
X_train_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_train.select_dtypes(exclude='number')))
X_train_cat_encoded.columns = X_train.select_dtypes(exclude='number').columns

X_test_cat_encoded = pd.DataFrame(ordinal_encoder.transform(X_test.select_dtypes(exclude='number')))
X_test_cat_encoded.columns = X_test.select_dtypes(exclude='number').columns

## Encoding the Independent Variable

In [5]:
label_encoder = LabelEncoder()
Y_train_cat_encoded= pd.DataFrame(label_encoder.fit_transform(Y_train))
Y_test_cat_encoded = pd.DataFrame(label_encoder.transform(Y_test))

## Standardization

In [6]:
sc = StandardScaler()
X_train_sc=pd.DataFrame(sc.fit_transform(X_train.select_dtypes(exclude='O')))
X_test_sc=pd.DataFrame(sc.transform(X_test.select_dtypes(exclude='O')))

X_train_sc.columns=X_train.select_dtypes(exclude='O').columns
X_test_sc.columns=X_test.select_dtypes(exclude='O').columns

## Combining data

In [7]:
X_train_final=pd.concat([X_train_sc,X_train_cat_encoded],axis=1)
X_test_final=pd.concat([X_test_sc,X_test_cat_encoded],axis=1)

## Handling imbalanced Dataset
### Since the dataset is small, will use over-sampling: SMOTE technique to balance the data

In [8]:
X_train_resample,Y_train_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_train_final,Y_train_cat_encoded)
X_test_resample,Y_test_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_test_final,Y_test_cat_encoded)

X_train_resample.shape,X_test_resample.shape,Y_train_resample.shape,Y_test_resample.shape

((7852, 28), (1476, 28), (7852, 1), (1476, 1))

## Feature Selection

In [9]:
print('Training dataset shape:', X_train_resample.shape, Y_train_resample.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample.shape)

Y_train_resample_flat = Y_train_resample.to_numpy().ravel()
Y_test_resample_flat = Y_test_resample.to_numpy().ravel()

print('Training dataset shape:', X_train_resample.shape, Y_train_resample_flat.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample_flat.shape)

Training dataset shape: (7852, 28) (7852, 1)
Testing dataset shape: (1476, 28) (1476, 1)
Training dataset shape: (7852, 28) (7852,)
Testing dataset shape: (1476, 28) (1476,)


### Forward selection approach

In [10]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
forward_fs = sfs(rf , k_features=10,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
forward_fs = forward_fs.fit(X_train_resample, Y_train_resample_flat)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:   49.7s finished

[2021-12-29 14:33:27] Features: 1/10 -- score: 0.7929174082798495[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   59.1s finished

[2021-12-29 14:34:27] Features: 2/10 -- score: 0.9684151844538956[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:   48.3s finished

[2021-12-29 14:35:15] Features: 3/10 -- score: 0.9859912344362591[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [11]:
# Create the Handler for logging records/messages to a file
file_handler = logging.FileHandler("log_file.log")

In [12]:
#set the format of the log records and the logging level to DEBUG
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG)

In [13]:
# a function  to create and save logs in the log files
def log(path, file):
    """[Create a log file to record the experiment's logs]
    
    Arguments:
        path {string} -- path to the directory
        file {string} -- file name
    
    Returns:
        [obj] -- [logger that record logs]
    """

    # check if the file exist
    log_file = os.path.join(path, file)

    if not os.path.isfile(log_file):
        open(log_file, "w+").close()

    console_logging_format = "%(levelname)s %(message)s"
    file_logging_format = "%(levelname)s: %(asctime)s: %(message)s"

    # configure logger
    logging.basicConfig(level=logging.INFO, format=console_logging_format)
    logger = logging.getLogger()
    
    # create a file handler for output file
    handler = logging.FileHandler(log_file)

    # set the logging level for log file
    handler.setLevel(logging.INFO)
    
    # create a logging format
    formatter = logging.Formatter(file_logging_format)
    handler.setFormatter(formatter)

    # add the handlers to the logger
    logger.addHandler(handler)

    return logger

In [14]:
logger = log(path=".",file="log_file.log")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
logger.info("Feature Selection")

2021-12-29 14:42:22,063 INFO:Feature Selection


In [16]:
feat_names = list(forward_fs.k_feature_names_)
logger.info("Features {}".format(feat_names))
X_train_new=X_train_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]
X_test_new=X_test_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]

2021-12-29 14:42:22,193 INFO:Features ['TSH', 'TT4', 'FTI', 'on_thyroxine', 'query_on_thyroxine', 'pregnant', 'goitre', 'psych', 'T3_measured', 'referral_source']


In [17]:
#Fitting the Random Forest model
rf_model=rf.fit(X_train_new,Y_train_resample_flat)

logger.info("Traininig Random Forest Model")

#Checking the metrics of Random Forest
def print_Score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        pred=clf.predict(x_train)
        clf_report=pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("-----------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
        logger.info("Train Result:\n==============="+"\n"+
                    f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%""\n"+
        "---------------------------------"+"\n"+
        f"Classification Report:\n{clf_report}"+"\n"+
        "-----------------------------------"+"\n"+
        f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
    elif train==False:
        pred=clf.predict(x_test)
        clf_report=pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("---------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")
        
        logger.info("Test Result:\n==============="+"\n"+
        f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%"+"\n"+
        "---------------------------------"+"\n"+
        f"Classification Report:\n{clf_report}"+"\n"+
        "---------------------------------"+"\n"+
        f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")
        
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2021-12-29 14:42:22,922 INFO:Traininig Random Forest Model
2021-12-29 14:42:23,074 INFO:Train Result:
Accuracy Score:99.73%
---------------------------------
Classification Report:
                     0            1            2       3  accuracy  \
precision     0.990914     1.000000     0.998474     1.0  0.997326   
recall        1.000000     0.989302     1.000000     1.0  0.997326   
f1-score      0.995436     0.994622     0.999236     1.0  0.997326   
support    1963.000000  1963.000000  1963.000000  1963.0  0.997326   

             macro avg  weighted avg  
precision     0.997347      0.997347  
recall        0.997326      0.997326  
f1-score      0.997324      0.997324  
support    7852.000000   7852.000000  
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [  18 1942    3    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]



Train Result:
Accuracy Score:99.73%
---------------------------------
Classification Report:
                     0            1            2       3  accuracy  \
precision     0.990914     1.000000     0.998474     1.0  0.997326   
recall        1.000000     0.989302     1.000000     1.0  0.997326   
f1-score      0.995436     0.994622     0.999236     1.0  0.997326   
support    1963.000000  1963.000000  1963.000000  1963.0  0.997326   

             macro avg  weighted avg  
precision     0.997347      0.997347  
recall        0.997326      0.997326  
f1-score      0.997324      0.997324  
support    7852.000000   7852.000000  
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [  18 1942    3    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2021-12-29 14:42:23,266 INFO:Test Result:
Accuracy Score:97.63%
---------------------------------
Classification Report:
                    0           1           2    3  accuracy    macro avg  \
precision    0.935361    1.000000    1.000000  0.0  0.976287     0.733840   
recall       1.000000    0.985772    0.943089  0.0  0.976287     0.732215   
f1-score     0.966601    0.992835    0.970711  0.0  0.976287     0.732537   
support    492.000000  492.000000  492.000000  0.0  0.976287  1476.000000   

           weighted avg  
precision      0.978454  
recall         0.976287  
f1-score       0.976716  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0   0

Test Result:
Accuracy Score:97.63%
---------------------------------
Classification Report:
                    0           1           2    3  accuracy    macro avg  \
precision    0.935361    1.000000    1.000000  0.0  0.976287     0.733840   
recall       1.000000    0.985772    0.943089  0.0  0.976287     0.732215   
f1-score     0.966601    0.992835    0.970711  0.0  0.976287     0.732537   
support    492.000000  492.000000  492.000000  0.0  0.976287  1476.000000   

           weighted avg  
precision      0.978454  
recall         0.976287  
f1-score       0.976716  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0   0]
 [  6 485   0   1]
 [ 28   0 464   0]
 [  0   0   0   0]]



In [18]:
##Hyper parameter tuning
RF=RandomForestClassifier()
model=RF.fit(X_train_new,Y_train_resample_flat)

print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2021-12-29 14:42:24,480 INFO:Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

2021-12-29 14:42:24,580 INFO:Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-scor

Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-score     0.972332    0.997963    0.972860   0.98103     0.981

In [19]:
## Randomized Search CV

#No of trees in Random Forest
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
#No of features consider at every split
max_features=['auto','sqrt','log2']
#maximum no of levels in trees
max_depth=[int(x) for x in np.linspace(10,1000,10)]
#minimum no of samples required to split a node
min_samples_split=[1,3,4,5,7,9]
#minimum samples leafs required at each leaf node
min_sample_leafs=[1,2,4,6,8]

#create random gird
random_grid={'n_estimators':n_estimators,
'max_features':max_features,
'max_depth':max_depth,
'min_samples_split':min_samples_split,
'min_samples_leaf':min_sample_leafs,
'criterion':['entropy','gini']}
print(random_grid)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [20]:
rcv=RandomizedSearchCV(estimator=RF,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=0,n_jobs=-1)

rcv.fit(X_train_new,Y_train_resample_flat)

logger.info("Use Randomized Search CV")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.0min finished
2021-12-29 14:49:28,879 INFO:Use Randomized Search CV


In [21]:
rcv.best_estimator_
logger.info(rcv.best_estimator_)

2021-12-29 14:49:28,895 INFO:RandomForestClassifier(criterion='entropy', max_depth=560, max_features='sqrt',
                       min_samples_split=9, n_estimators=400)


In [22]:
best_random_grid=rcv.best_estimator_
logger.info("Result with best estimetors")
print_Score(best_random_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2021-12-29 14:49:29,009 INFO:Result with best estimetors
2021-12-29 14:49:29,201 INFO:Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [23]:
param_grid = {
    'criterion': [rcv.best_params_['criterion']],
    'max_depth': [rcv.best_params_['max_depth']],
    'max_features': [rcv.best_params_['max_features']],
    'min_samples_leaf': [rcv.best_params_['min_samples_leaf'], 
                         rcv.best_params_['min_samples_leaf']+2, 
                         rcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rcv.best_params_['min_samples_split'] - 2,
                          rcv.best_params_['min_samples_split'] - 1,
                          rcv.best_params_['min_samples_split'], 
                          rcv.best_params_['min_samples_split'] +1,
                          rcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rcv.best_params_['n_estimators'] - 200, rcv.best_params_['n_estimators'] - 100, 
                     rcv.best_params_['n_estimators'], 
                     rcv.best_params_['n_estimators'] + 100, rcv.best_params_['n_estimators'] + 200]
}

In [24]:
print(param_grid)
logger.info(f"Parameter Grid: {param_grid}")

2021-12-29 14:49:29,245 INFO:Parameter Grid: {'criterion': ['entropy'], 'max_depth': [560], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [7, 8, 9, 10, 11], 'n_estimators': [200, 300, 400, 500, 600]}


{'criterion': ['entropy'], 'max_depth': [560], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [7, 8, 9, 10, 11], 'n_estimators': [200, 300, 400, 500, 600]}


In [25]:
grid_search=GridSearchCV(estimator=RF,param_grid=param_grid,cv=10,n_jobs=1,verbose=2)
grid_search.fit(X_train_new,Y_train_resample_flat)
logger.info("Grid Search CV: ")

Fitting 10 folds for each of 75 candidates, totalling 750 fits
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, total=   2.6s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, total=   2.4s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, total=   2.1s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, total=   1.9s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, total=   2.4s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200 
[CV]

...

[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=400, total=   6.0s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=400 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=400, total=   5.9s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=500 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=500, total=   8.0s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=500 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=500, total=   7.6s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, n_estimators=500 
[CV]

[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   4.7s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   4.8s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   4.6s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   4.3s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=3

[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600, total=   8.3s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600, total=   9.2s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600, total=   9.2s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=600, total=   9.4s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=6

[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=400, total=   6.3s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=400 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=400, total=   6.1s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=400 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=400, total=   6.6s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=500 
[CV]  criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=500, total=   7.9s
[CV] criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=5, min_samples_split=11, n_estimators=5

[Parallel(n_jobs=1)]: Done 750 out of 750 | elapsed: 72.4min finished
2021-12-29 16:02:02,281 INFO:Grid Search CV: 


In [26]:
best_grid=grid_search.best_estimator_
best_grid
logger.info(f"Best Grid: {best_grid}")

2021-12-29 16:02:02,311 INFO:Best Grid: RandomForestClassifier(criterion='entropy', max_depth=560, max_features='sqrt',
                       min_samples_split=7, n_estimators=500)


In [27]:
logger.info("Result with Best Grid: ")
print_Score(best_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2021-12-29 16:02:02,342 INFO:Result with Best Grid: 
2021-12-29 16:02:02,596 INFO:Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-score     0.972332    0.997963    0.972860   0.98103     0.981052   
support    492.000000  492.000000  492.000000   0.98103  1476.000000   

           weighted avg  
precision      0.982051  
recall         0.981030  
f1-score       0.981052  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 26   0 466]]



Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-score     0.972332    0.997963    0.972860   0.98103     0.981052   
support    492.000000  492.000000  492.000000   0.98103  1476.000000   

           weighted avg  
precision      0.982051  
recall         0.981030  
f1-score       0.981052  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 26   0 466]]



In [28]:
#Now, will convert our final model into pickle file

import pickle

pickle_out=open('Thyroid_model.pkl','wb')
pickle.dump(grid_search,pickle_out)
pickle_out.close()
logger.info("Random Forest Model with Grid Search Saved in Pickle")

2021-12-29 16:02:02,667 INFO:Random Forest Model with Grid Search Saved in Pickle
